In [1]:
from inference.gp_tools import GpOptimiser

import matplotlib.pyplot as plt
import matplotlib as mpl
from numpy import sin, cos, linspace, array, meshgrid

mpl.rcParams['axes.autolimit_mode'] = 'round_numbers'
mpl.rcParams['axes.xmargin'] = 0
mpl.rcParams['axes.ymargin'] = 0
"""
GpOptimiser extends the functionality of GpRegressor to perform 'Bayesian optimisation'.

Bayesian optimisation is suited to problems for which a single evaluation of the function
being explored is expensive, such that the total number of function evaluations must be
made as small as possible.
"""

# define the function whose maximum we will search for
def search_function(x):
    return sin(0.5*x) + 3 / (1 + (x-1)**2)

# define bounds for the optimisation
bounds = [(-8,8)]

# create some initialisation data
x = array([-8,-7,6,8])
y = search_function(x)

# create an instance of GpOptimiser
GP = GpOptimiser(x,y,bounds=bounds)


# here we evaluate the search function for plotting purposes
M = 500
x_gp = linspace(*bounds[0],M)
y_func = search_function(x_gp)
mu, sig = GP(x_gp)
means = [mu]
sigmas = [sig]
acquis = [array([GP.acquisition(k) for k in x_gp])]
max_values = [max(GP.y)]
evaluations = [len(GP.y)]


for i in range(7):
    # request the proposed evaluation
    new_x = GP.propose_evaluation()

    # evaluate the new point
    new_y = search_function(new_x)

    # update the gaussian process with the new information
    GP.add_evaluation(new_x, new_y)

    # store some data for plotting the iterations
    mu, sig = GP(x_gp)
    means.append(mu)
    sigmas.append(sig)
    acquis.append(array([GP.acquisition(k) for k in x_gp]))
    max_values.append(max(GP.y))
    evaluations.append(len(GP.y))
    

    

In [2]:
# L = len(means)//2
# fig, axes = plt.subplots(4, L, gridspec_kw={'height_ratios': [3, 1, 3, 1]}, figsize = (16,10))
# for i in range(L):
#     for j in range(2):
#         k = i+4*j
#         axes[0+2*j,i].plot(GP.x[:len(GP.x)-2*L+k], GP.y[:len(GP.x)-2*L+k], 'o', c = 'red', label = 'observations', zorder = 5)
#         axes[0+2*j,i].plot(x_gp, y_func, lw = 1.5, c = 'red', ls = 'dashed', label = 'actual function')
#         axes[0+2*j,i].plot(x_gp, means[k], lw = 2, c = 'blue', label = 'GP prediction')
#         axes[0+2*j,i].fill_between(x_gp, (means[k]-2*sigmas[k]), y2=(means[k]+2*sigmas[k]), color = 'blue', alpha = 0.15, label = '95% confidence interval')
#         axes[0+2*j,i].set_ylim([-1.5,4])
#         axes[0+2*j,i].set_xticks([])

#         aq = acquis[i]
#         proposal = x_gp[aq.argmax()]
#         axes[1+2*j,i].fill_between(x_gp, 0.9*aq/aq.max(), color = 'green', alpha = 0.15)
#         axes[1+2*j,i].plot(x_gp, 0.9*aq/aq.max(), color = 'green', label = 'acquisition function')
#         axes[1+2*j,i].plot([proposal]*2, [0.,1.], c = 'green', ls = 'dashed', label = 'acquisition maximum')
#         axes[0+2*j,i].plot([proposal]*2, [-1.5,search_function(proposal)], c = 'green', ls = 'dashed')
#         axes[0+2*j,i].plot(proposal, search_function(proposal), 'o', c = 'green', label = 'proposed observation')
#         axes[1+2*j,i].set_ylim([0,1])
#         axes[1+2*j,i].set_xlim([-8,8])
#         axes[1+2*j,i].set_yticks([])
#         axes[1+2*j,i].set_xlabel('x')
#     #     axes[1,i].legend(loc=1)
#     #     axes[0,i].legend(loc=2)

#         if i > 0: 
#             axes[0+2*j,i].set_yticks([])
#         else:
#             axes[0+2*j,i].set_ylabel('y')
        

# plt.tight_layout()
# plt.subplots_adjust(hspace=0)
# plt.subplots_adjust(wspace=0.05)
# plt.show()